<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/attrition.png?raw=true" width=800 align="left"/>

# 실습 내용

- Attrition 데이터로 모델링합니다.
- KNN 알고리즘으로 모델링합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [145]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format='retina'

In [146]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Attrition_simple2.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [147]:
# 상위 몇 개 행 확인
data.head()

,Attrition,Age,DistanceFromHome,EmployeeNumber,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,817,Male,3,Married,11691,No,11,14
1,0,35,18,1412,Male,4,Single,9362,No,11,10
2,0,42,6,1911,Male,1,Married,13348,No,13,18
3,0,46,2,1204,Female,1,Married,17048,No,23,28
4,1,22,4,593,Male,3,Single,3894,No,16,4


**데이터 설명**

- Attrition: 이직 여부 (1: 이직, 0: 잔류)
- Age: 나이
- DistanceFromHome: 집-직장 거리 (단위: 마일)
- EmployeeNumber: 사번	
- Gender: 성별 (Male, Female)
- JobSatisfaction: 직무 만족도(1: Low, 2: Medium, 3: High, 4: Very High)
- MaritalStatus: 결혼 상태 (Single, Married, Divorced)
- MonthlyIncome: 월급 (단위: 달러)
- OverTime: 야근 여부 (Yes, No)
- PercentSalaryHike: 전년 대비 급여 인상율(단위: %)
- TotalWorkingYears: 총 경력 연수

In [148]:
# 기초통계량 확인
data.describe()

,Attrition,Age,DistanceFromHome,EmployeeNumber,JobSatisfaction,MonthlyIncome,PercentSalaryHike,TotalWorkingYears
count,1196.000000,1196.00000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000
mean,0.163043,36.94398,9.258361,1035.629599,2.716555,6520.104515,15.251672,11.330268
std,0.369560,9.09270,8.166016,604.340130,1.110962,4665.902253,3.625946,7.823821
min,0.000000,18.00000,1.000000,1.000000,1.000000,1009.000000,11.000000,0.000000
25%,0.000000,30.00000,2.000000,507.750000,2.000000,2928.250000,12.000000,6.000000
50%,0.000000,36.00000,7.000000,1028.000000,3.000000,4973.500000,14.000000,10.000000
75%,0.000000,43.00000,14.000000,1581.250000,4.000000,8420.500000,18.000000,15.000000
max,1.000000,60.00000,29.000000,2068.000000,4.000000,19999.000000,25.000000,40.000000


In [149]:
# NaN 값 확인
data.isnull().sum()

Attrition            0
Age                  0
DistanceFromHome     0
EmployeeNumber       0
Gender               0
JobSatisfaction      0
MaritalStatus        0
MonthlyIncome        0
OverTime             0
PercentSalaryHike    0
TotalWorkingYears    0
dtype: int64

In [150]:
# 상관관계 확인
data.corr()

,Attrition,Age,DistanceFromHome,EmployeeNumber,JobSatisfaction,MonthlyIncome,PercentSalaryHike,TotalWorkingYears
Attrition,1.000000,-0.167866,0.081973,-0.008707,-0.078936,-0.163572,-0.000048,-0.182162
Age,-0.167866,1.000000,-0.010917,-0.023786,-0.012425,0.490107,-0.008303,0.674331
DistanceFromHome,0.081973,-0.010917,1.000000,0.054948,-0.021623,-0.012803,0.052348,0.002606
EmployeeNumber,-0.008707,-0.023786,0.054948,1.000000,-0.022863,-0.014032,-0.009514,-0.016317
JobSatisfaction,-0.078936,-0.012425,-0.021623,-0.022863,1.000000,-0.025082,0.030811,-0.039380
MonthlyIncome,-0.163572,0.490107,-0.012803,-0.014032,-0.025082,1.000000,-0.021334,0.768437
PercentSalaryHike,-0.000048,-0.008303,0.052348,-0.009514,0.030811,-0.021334,1.000000,-0.021988
TotalWorkingYears,-0.182162,0.674331,0.002606,-0.016317,-0.039380,0.768437,-0.021988,1.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 변수 제거**

- 제거 대상 변수: EmployeeNumber

In [151]:
# 제거 대상: EmployeeNumber

data.drop('EmployeeNumber' , axis=1 , inplace =True)
# 변수 제거


# 확인
data.head()

,Attrition,Age,DistanceFromHome,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,Male,3,Married,11691,No,11,14
1,0,35,18,Male,4,Single,9362,No,11,10
2,0,42,6,Male,1,Married,13348,No,13,18
3,0,46,2,Female,1,Married,17048,No,23,28
4,1,22,4,Male,3,Single,3894,No,16,4


**2) x, y 분리**

In [152]:
# target 확인
target = 'Attrition'

# 데이터 분리
x = data.drop(target, axis = 1)
y = data[target]


**3) 가변수화**

In [153]:
# 가변수화 대상: Gender, JobSatisfaction, MaritalStatus, OverTime
dumm_cols = ['Gender', 'JobSatisfaction', 'MaritalStatus', 'OverTime']

# 가변수화
x = pd.get_dummies(data = x, columns = dumm_cols, drop_first = True)

# 확인
x.head()

,Age,DistanceFromHome,MonthlyIncome,PercentSalaryHike,TotalWorkingYears,Gender_Male,JobSatisfaction_2,JobSatisfaction_3,JobSatisfaction_4,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,33,7,11691,11,14,1,0,1,0,1,0,0
1,35,18,9362,11,10,1,0,0,1,0,1,0
2,42,6,13348,13,18,1,0,0,0,1,0,0
3,46,2,17048,23,28,0,0,0,0,1,0,0
4,22,4,3894,16,4,1,0,1,0,0,1,0


**4) 정규화**

In [154]:
# 정규화 대상: Age, DistanceFromHome, MonthlyIncome, PercentSalaryHike, TotalWorkingYears
norm_cols = ['Age', 'DistanceFromHome', 'MonthlyIncome', 
             'PercentSalaryHike', 'TotalWorkingYears']

# 부분 정규화
x[norm_cols] = (x[norm_cols] - x[norm_cols].min()) / (x[norm_cols].max() - x[norm_cols].min())


# 전부 정규화
# x = (x - x.min()) / (x.max() - x.min())

# 확인
x.head()

,Age,DistanceFromHome,MonthlyIncome,PercentSalaryHike,TotalWorkingYears,Gender_Male,JobSatisfaction_2,JobSatisfaction_3,JobSatisfaction_4,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,0.357143,0.214286,0.562507,0.000000,0.35,1,0,1,0,1,0,0
1,0.404762,0.607143,0.439863,0.000000,0.25,1,0,0,1,0,1,0
2,0.571429,0.178571,0.649763,0.142857,0.45,1,0,0,0,1,0,0
3,0.666667,0.035714,0.844602,0.857143,0.70,0,0,0,0,1,0,0
4,0.095238,0.107143,0.151922,0.357143,0.10,1,0,1,0,0,1,0


**5) 학습용, 평가용 데이터 분리**

In [155]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.

In [156]:
# 1단계: 불러오기
from sklearn.neighbors import KNeighborsClassifier


In [157]:
# 2단계: 선언하기
model = KNeighborsClassifier(n_neighbors=5)

In [158]:
# 3단계: 학습하기
model.fit(x_train,y_train)

KNeighborsClassifier()

In [159]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [160]:
# 5단계: 평가하기
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[286  14]
 [ 45  14]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91       300
           1       0.50      0.24      0.32        59

    accuracy                           0.84       359
   macro avg       0.68      0.60      0.61       359
weighted avg       0.80      0.84      0.81       359



In [161]:
for i in range(1,11,2) :
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print()
    print(f' n = {i} --------------------------------------------')
    print()
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))


 n = 1 --------------------------------------------

[[267  33]
 [ 44  15]]
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       300
           1       0.31      0.25      0.28        59

    accuracy                           0.79       359
   macro avg       0.59      0.57      0.58       359
weighted avg       0.77      0.79      0.78       359


 n = 3 --------------------------------------------

[[287  13]
 [ 47  12]]
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       300
           1       0.48      0.20      0.29        59

    accuracy                           0.83       359
   macro avg       0.67      0.58      0.60       359
weighted avg       0.80      0.83      0.80       359


 n = 5 --------------------------------------------

[[286  14]
 [ 45  14]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91       300
      

In [163]:
# 표본이 너무작은 것에 대해 확률을 따지는건 무의미 할 수 있다.
y.value_counts()


0    1001
1     195
Name: Attrition, dtype: int64